In [42]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import settrade_v2
from settrade_v2 import Investor
import time

In [43]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

WARNING	cassandra.cluster:cluster.py:__init__()- Cluster.__init__ called with contact_points specified, but no load_balancing_policy. In the next major version, this will raise an error; please specify a load-balancing policy. (contact_points = ['127.0.0.1'], lbp = None)
WARNING	cassandra.cluster:cluster.py:protocol_downgrade()- Downgrading core protocol version from 66 to 65 for 127.0.0.1:9042. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version
WARNING	cassandra.cluster:cluster.py:protocol_downgrade()- Downgrading core protocol version from 65 to 5 for 127.0.0.1:9042. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version


In [44]:
import settrade_v2
from settrade_v2 import Investor

investor = Investor(
    app_id="atBqrBrphDcChF32",                                 
    app_secret="AK0WsjfbWrRMIL9aCi8b7i12hGdEojdohbUCCb++jDTe", 
    broker_id="SANDBOX",
    app_code="SANDBOX",
    is_auto_queue=False
)

deri = investor.Derivatives(account_no="Man-D")            

account_info = deri.get_account_info()

account_info    

INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.263802s]login - success
INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.384236s]get_account_info - success


{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [45]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="AOT",
interval="1d",
limit=1,
normalized=True,)

INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.588140s]get_candlestick - success


In [46]:
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,0,1760634000,40.0,41.5,39.75,40.25,38939890,0


In [47]:
session.set_keyspace('data_stock')

In [ ]:
# ✅ โหลดรายชื่อหุ้นทั้งหมด
import pandas as pd

try:
    symbols_df = pd.read_excel("/Users/hilmanyusoh/Desktop/Stock-market-chart-pattern-recognition-using-deep-learning/data_collection/settrade1.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'get_stock_name' กรุณาตรวจสอบชื่อไฟล์และตำแหน่งให้ถูกต้อง")
    raise  # หยุดโปรแกรมทันทีโดยแสดง error


# ✅ ฟังก์ชันดึง candlestick จาก API
def get_candlestick_data(symbol):
    try:
        res = market.get_candlestick(  # ✅ ต้องกำหนด market ก่อนใช้
            symbol=symbol,
            interval="1d",
            limit=1000,
            normalized=True
        )
        return res
    except Exception as e:
        print(f"❌ ไม่สามารถดึงข้อมูลของ {symbol}: {e}")
        return None

# ✅ ฟังก์ชัน insert ข้อมูลลง Cassandra
def insert_historical_data(symbol, res):
    if not res or 'time' not in res:
        print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}")
        return

    for i in range(len(res["time"])):
        # ✅ แปลง timestamp ให้ถูกต้อง
        timestamp = datetime.fromtimestamp(res["time"][i])

        session.execute("""
            INSERT INTO candlestick_data (
                symbol, time, open_price, high_price, low_price,
                close_price, volume, value
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            symbol,
            timestamp,
            res["open"][i],
            res["high"][i],
            res["low"][i],
            res["close"][i],
            res["volume"][i],
            res["value"][i]
        ))

    print(f"✅ เพิ่มข้อมูล {len(res['time'])} รายการของหุ้น {symbol} สำเร็จ!")

# ✅ ดึงและบันทึกข้อมูลทุกหุ้น
for symbol in symbols:
    print(f"\n📦 กำลังโหลดข้อมูล {symbol} ...")
    res = get_candlestick_data(symbol)
    insert_historical_data(symbol, res)
    time.sleep(2)  # 🕒 ลดโอกาสถูก block จาก API

print("\n🎉 เสร็จสิ้นการโหลดข้อมูลทั้งหมด!")


📦 กำลังโหลดข้อมูล 24CS ...


INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.340462s]get_candlestick - success


✅ เพิ่มข้อมูล 731 รายการของหุ้น 24CS สำเร็จ!

📦 กำลังโหลดข้อมูล 2S ...


INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.208146s]get_candlestick - success


✅ เพิ่มข้อมูล 731 รายการของหุ้น 2S สำเร็จ!

📦 กำลังโหลดข้อมูล 3BBIF ...


...

INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.314114s]get_candlestick - success


✅ เพิ่มข้อมูล 731 รายการของหุ้น ASK สำเร็จ!

📦 กำลังโหลดข้อมูล ASN ...


INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.287698s]get_candlestick - success


✅ เพิ่มข้อมูล 721 รายการของหุ้น ASN สำเร็จ!

📦 กำลังโหลดข้อมูล ASP ...


INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.358421s]get_candlestick - success


In [ ]:
stock = "AOT"
rows = session.execute(f"SELECT * FROM candlestick_data WHERE symbol = '{stock}'")

In [ ]:
df = pd.DataFrame(rows, columns=[
    'symbol', 'time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'value'
])

# ✅ แปลงชนิดเวลา + เรียงลำดับ
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)
df.head()

,symbol,time,open_price,high_price,low_price,close_price,volume,value
0,AOT,2022-10-17,70.25,71.25,70.0,71.00,0.0,41596716
1,AOT,2022-10-18,72.00,72.25,70.0,70.25,0.0,32748153
2,AOT,2022-10-19,72.00,72.50,71.5,72.50,0.0,12560022
3,AOT,2022-10-20,73.00,73.00,71.0,71.50,0.0,26006293
4,AOT,2022-10-21,73.50,73.50,72.5,72.75,0.0,16760283


In [ ]:
import plotly.graph_objects as go

# ✅ สร้างกราฟ Candlestick
fig = go.Figure()

# 🔹 แท่งเทียนราคา
fig.add_trace(go.Candlestick(
    x=df['time'],
    open=df['open_price'],
    high=df['high_price'],
    low=df['low_price'],
    close=df['close_price'],
    name=stock
))

# 🔹 Volume bar (แถบปริมาณการซื้อขาย)
fig.add_trace(go.Bar(
    x=df['time'],
    y=df['volume'],
    marker_color='rgba(128,128,128,0.3)',
    yaxis='y2',
    name='Volume'
))

# ✅ ปรับ layout ให้เหมือน TradingView
fig.update_layout(
    title=f'📈 {stock} - Candlestick Chart with Volume',
    xaxis_rangeslider_visible=False,
    yaxis=dict(title='Price'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    height=600,
    template='plotly_white'
)

fig.show()